### Stored procedure testing

In [ ]:
-- CREATE PROCEDURE run_materialization
 --   @Workspace NVARCHAR(128),
 --   @medallion_name NVARCHAR(128),
 --   @InputSourceSchema NVARCHAR(128),
 --   @InputSource NVARCHAR(128),
 --   @OutputTargetSchema NVARCHAR(128),
 --   @OutputTarget NVARCHAR(128),
 --   @UpdateControlTable BIT = 0,
 --   @LoadOption NVARCHAR(10)
--AS

BEGIN
    SET NOCOUNT ON;
    DECLARE @Workspace VARCHAR(128) = 'BUNN_Foundation_NONPROD';
    DECLARE @medallion_name VARCHAR(128) = 'gold_spaecc_warehouse';
    DECLARE @InputSourceSchema VARCHAR(128) = 'materialized_etl';
    DECLARE @InputSource VARCHAR(128) = 'acct_doc_hdr_etl';
    DECLARE @OutputTargetSchema VARCHAR(128) = 'materialized_t';
    DECLARE @OutputTarget VARCHAR(128) = 'acct_doc_hdr';
    DECLARE @UpdateControlTable BIT = 0;
    DECLARE @LoadOption VARCHAR(10) = 'TR';

    DECLARE @RunStart DATETIME = sysdatetime();
    DECLARE @ControlStartTimestamp DATETIME = '1900-01-01 00:00:00';
    DECLARE @SourceTablePath VARCHAR(MAX);
    DECLARE @TargetTablePath VARCHAR(MAX);
    DECLARE @LogTablePath VARCHAR(MAX);
    DECLARE @SQLQuery NVARCHAR(MAX);
    DECLARE @ErrorMessage VARCHAR(4000);
    DECLARE @ErrorSeverity INT;
    DECLARE @ErrorState INT;

-- Begin Materialization Process
        INSERT INTO utilities.materialization_log (schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, statement_text, medallion_name)
        VALUES (@InputSourceSchema, @InputSource, @RunStart, 10, 'begin materialization', sysdatetime(), null, @medallion_name);

-- Update or Insert into materialization_control table
        IF @UpdateControlTable = 1
        BEGIN
            UPDATE utilities.materialization_control
            SET control_start_timestamp = sysdatetime()
            WHERE etl_database = @InputSourceSchema
              AND etl_view = @InputSource
              AND medallion_name = @medallion_name
              AND load_option = @LoadOption;
        END
        ELSE
        BEGIN
            INSERT INTO utilities.materialization_control (etl_database, etl_view, status_code, load_option, extract_start_timestamp, control_start_timestamp, medallion_name)
            SELECT @InputSourceSchema, @InputSource, 'active', @LoadOption, @RunStart, NULL, @medallion_name
            WHERE NOT EXISTS (
                SELECT 1
                FROM utilities.materialization_control
                WHERE etl_database = @InputSourceSchema
                  AND etl_view = @InputSource
                  AND load_option = @LoadOption
                  AND medallion_name = @medallion_name
            );
        END;

-- Update materialization_load table
        UPDATE utilities.materialization_load
        SET last_execution = sysdatetime()
        WHERE medallion_name = @medallion_name
          AND input_source_schema = @InputSourceSchema
          AND input_source = @InputSource
          AND output_target_schema = @OutputTargetSchema
          AND output_target = @OutputTarget
          AND load_option = @LoadOption;       
/*
===============================================================================================================
-- METADATA CHECK
=================================================================================================================
*/
-- Check if the source table exists
IF NOT EXISTS (SELECT 1 FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = @InputSourceSchema AND TABLE_NAME = @InputSource)
        BEGIN
            INSERT INTO materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 21, 'Failure: Source object validation', sysdatetime(), 1, 'True', 'Source table does not exist.');

            THROW 50001, 'Source table does not exist.', 1;
        END
        ELSE
        BEGIN
            INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 20, 'Success: Source object validation', sysdatetime(), NULL, NULL, 'Source table exists.');
        END;

-- Check if the target table exists

        IF NOT EXISTS (SELECT 1 FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = @OutputTargetSchema AND TABLE_NAME = @OutputTarget)
        BEGIN
            INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 31, 'Failure: Target object validation', sysdatetime(), 1, 1, 'Target table does not exist.');

            THROW 50002, 'Target table does not exist.', 1;
        END
        ELSE
        BEGIN
            INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 30, 'Success: Target object validation', sysdatetime(), NULL, NULL, 'Target table exists.');
        END;

-- Check column counts match

        DECLARE @SourceColumnCount INT;
        DECLARE @TargetColumnCount INT;

        SELECT @SourceColumnCount = COUNT(*)
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_SCHEMA = @InputSourceSchema AND TABLE_NAME = @InputSource;

        SELECT @TargetColumnCount = COUNT(*)
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_SCHEMA = @OutputTargetSchema AND TABLE_NAME = @OutputTarget;

        IF @SourceColumnCount <> @TargetColumnCount
        BEGIN
            INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 41, 'Failure: Column counts mismatch', sysdatetime(), 1, 1, 'Column count mismatch.');

            THROW 50003, 'Column count mismatch.', 1;
        END
        ELSE
        BEGIN
            INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 40, 'Success: Column counts match', sysdatetime(), NULL, NULL, 'Column counts match.');
        END;

-- Check column names match
        IF EXISTS (
            SELECT COLUMN_NAME
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA = @InputSourceSchema AND TABLE_NAME = @InputSource
            EXCEPT
            SELECT COLUMN_NAME
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA = @OutputTargetSchema AND TABLE_NAME = @OutputTarget
        )
        BEGIN
            INSERT INTO utiities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 51, 'Failure: Column names mismatch', sysdatetime(), 1, 1, 'Column names mismatch.');

            THROW 50004, 'Column names mismatch.', 1;
        END
        ELSE
        BEGIN
            INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 50, 'Success: Column names match', sysdatetime(), NULL, NULL, 'Column names match.');
        END;

-- Check column data types match
        IF EXISTS (
            SELECT COLUMN_NAME, DATA_TYPE
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA = @InputSourceSchema AND TABLE_NAME = @InputSource
            EXCEPT
            SELECT COLUMN_NAME, DATA_TYPE
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA = @OutputTargetSchema AND TABLE_NAME = @OutputTarget
        )
        BEGIN
            INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 61, 'Failure: Column data types mismatch', sysdatetime(), 1, 1, 'Column data types mismatch.');

            THROW 50005, 'Column data types mismatch.', 1;
        END
        ELSE
        BEGIN
            INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 60, 'Success: Column data types match', sysdatetime(), NULL, NULL, 'Column data types match.');
        END;
/*
================================================================================================================
-- Data Loads
================================================================================================================
*/
 -- Truncate and Reload
        IF @LoadOption = 'TR'
        BEGIN
            BEGIN TRY
                SET @SQLQuery = N'TRUNCATE TABLE ' + QUOTENAME(@OutputTargetSchema) + N'.' + QUOTENAME(@OutputTarget);
                EXEC sp_executesql @SQLQuery;

                INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
                VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 101, 'Success: Truncate the Table', sysdatetime(), NULL, NULL, 'Truncate the table completed.');
            END TRY
            BEGIN CATCH
                INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
                VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 191, 'Failure: Truncate the table', sysdatetime(), 1, 1, @SQLQuery);
                THROW 50006, 'Failure: Truncate the Table.', 1;
            END CATCH;
            BEGIN TRY
                SET @SQLQuery = N'INSERT INTO ' + QUOTENAME(@OutputTargetSchema) + N'.' + QUOTENAME(@OutputTarget) + 
                                N' SELECT * FROM ' + QUOTENAME(@InputSourceSchema) + N'.' + QUOTENAME(@InputSource);
                EXEC sp_executesql @SQLQuery;
            INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
                VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 102, 'Success: Truncate and Reload', sysdatetime(), NULL, NULL, 'Truncate and Reload completed.');
            END TRY
            BEGIN CATCH
                INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
                VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 192, 'Failure: Truncate and Reload', sysdatetime(), 1, 1, @SQLQuery);
                THROW 50007, 'Failure: Truncate and Reload.', 1;
            END CATCH;    
        END
        ELSE
-- Unsupported Load Option
        BEGIN
            INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
            VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 999, 'Failure: Unsupported Load Option', sysdatetime(), 1, 1, 'Unsupported load option.');
            THROW 50008, 'Failure: Unsupported Load Option.', 1;
        END;

-- Load Complete
         INSERT INTO utilities.materialization_log (medallion_name, schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail, statement_text)
        VALUES (@medallion_name, @InputSourceSchema, @InputSource, @RunStart, 200, 'Load Complete', sysdatetime(), NULL, NULL, 'Load completed successfully.');

        PRINT 'Materialization process completed successfully.';
    END;


